### Projet de laboratoire :  Système d'identification de visages

L'objectif de ce projet est la conception d'un système d'identification de visages robustes à partir d'images statiques (en niveau de gris)

Nous étudierons différents classificateurs (K-NN, SVM, CNNs), l'impacts de différents algorithmes de réduction de dimensionnalité (PCA et LDA) sur l'apprentissage et la performance des classificateurs, indépendamment mais aussi en les combinant.

### Importation de la dataset (lfw_pepole)


In [1]:
# Importation de la dataset lfw_people

from sklearn.datasets import fetch_lfw_people
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

### Différents algorithmes de réduction de dimensionnalité (PCA, LDA)


In [2]:
from sklearn.model_selection import train_test_split

# Stratification des données train (60%), val (30%), test (10%)

X = lfw_people.data
n_features = X.shape[1]
y = lfw_people.target

# Conservation de la stratification trainval (90%) et test (10%) pour k-fold cross validation

X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.10, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.30, random_state=42)

In [3]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Entrainement de l'algorithme PCA

n_components_PCA = 100
pca = PCA(n_components=n_components_PCA).fit(X_train)

# Entrainement de l'algorithme LDA

n_components_LDA = 4
lda = LDA(n_components=n_components_LDA).fit(X_train, y_train)

In [4]:
# Projection des visages dans un espace de dimension plus faible

# PCA
X_train_pca = pca.transform(X_train)
X_val_pca = pca.transform(X_val)
X_test_pca = pca.transform(X_test)

# LDA
X_train_lda = lda.transform(X_train)
X_val_lda = lda.transform(X_val)
X_test_lda = lda.transform(X_test)

# Classification brute avec LDA
LDA_val_score = lda.score(X_val, y_val)
LDA_test_score = lda.score(X_test, y_test)

print(f"Taux de bonnes classification validation - n_components={n_components_LDA} : {LDA_val_score:0.2%}")
print(f"Taux de bonnes classification test - n_components={n_components_LDA} : {LDA_test_score:0.2%}")

Taux de bonnes classification validation - n_components=4 : 72.41%
Taux de bonnes classification test - n_components=4 : 78.29%


### Combinaison des réducteurs et des classificateurs


In [5]:
# PCA puis LDA

lda_pcatrained = LDA(n_components=n_components_LDA).fit(X_train_pca, y_train)
X_train_pca_lda = lda_pcatrained.transform(X_train_pca)
X_val_pca_lda = lda_pcatrained.transform(X_val_pca)
X_test_pca_lda = lda_pcatrained.transform(X_test_pca)

# Test immédiat sur une classification brute avec LDA
LDA_pca_val_score = lda_pcatrained.score(X_val_pca, y_val)
print(f"Taux de bonnes classification validation - n_components={n_components_LDA} : {LDA_pca_val_score:0.2%}")
# Meilleur classification que LDA seul

Taux de bonnes classification validation - n_components=4 : 82.18%


### Différents classificateurs indépendants

#### K-nearest neighbors

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# Entrainement de l'algorithme KNN

neigh = KNeighborsClassifier(n_neighbors=12) # Best k (based on validation)
neigh.fit(X_train, y_train) # Entraînement du modèle

KNN_val_score = neigh.score(X_val, y_val)
KNN_test_score = neigh.score(X_test, y_test)

# Cross validation score
KNN_cross_val_score = cross_val_score(neigh,X_val, y_val, cv = 4)
KNN_cross_test_score = cross_val_score(neigh,X_test, y_test, cv = 4)

print("RAW:",KNN_val_score)
print("Cross-validation:",max(KNN_cross_val_score)) # index = 2

# Entrainement de l'algorithme KNN avec PCA

neigh_PCA = KNeighborsClassifier(n_neighbors=12)
neigh_PCA.fit(X_train_pca,y_train)

KNN_val_score_PCA = neigh_PCA.score(X_val_pca, y_val)
KNN_test_score_PCA = neigh_PCA.score(X_test_pca, y_test)
KNN_cross_val_score_PCA = cross_val_score(neigh_PCA,X_val_pca, y_val, cv = 4)
KNN_cross_test_score_PCA = cross_val_score(neigh_PCA,X_test_pca, y_test, cv = 4)

print("PCA:",KNN_val_score_PCA)
print("Cross-validation_PCA:",max(KNN_cross_val_score_PCA)) 

# Entrainement de l'algorithme KNN avec LDA

neigh_LDA = KNeighborsClassifier(n_neighbors=12)
neigh_LDA.fit(X_train_lda,y_train)

KNN_val_score_LDA = neigh_LDA.score(X_val_lda, y_val)
KNN_test_score_LDA = neigh_LDA.score(X_test_lda, y_test)
KNN_cross_val_score_LDA = cross_val_score(neigh_LDA,X_val_lda, y_val, cv = 4)
KNN_cross_test_score_LDA = cross_val_score(neigh_LDA,X_test_lda, y_test, cv = 4)

print("LDA:",KNN_val_score_LDA)
print("Cross-validation_LDA:",max(KNN_cross_val_score_LDA)) 

# Entrainement de l'algorithme KNN avec PCA + LDA

neigh_PCA_LDA = KNeighborsClassifier(n_neighbors=12)
neigh_PCA_LDA.fit(X_train_lda,y_train)

KNN_val_score_PCA_LDA = neigh_LDA.score(X_val_pca_lda, y_val)
KNN_test_score_PCA_LDA = neigh_LDA.score(X_test_pca_lda, y_test)

print("PCA + LDA:",KNN_val_score_PCA_LDA)


RAW: 0.514367816091954
Cross-validation: 0.5402298850574713
PCA: 0.5344827586206896
Cross-validation_PCA: 0.5402298850574713
LDA: 0.6982758620689655
Cross-validation_LDA: 0.7126436781609196
PCA + LDA: 0.5373563218390804


### Support Vector Machine (SVM)

In [7]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold

# K-fold crossvalidation avec K = 4
kf = KFold(n_splits=4)
kf.get_n_splits(X_trainval)

# Choix du meilleur coefficient de soft margin
c = [0.001, 0.01, 0.1, 1, 5, 10, 100, 1000]
kern = "linear"

# PCA ou LDA puis linear SVM
for C in c : 
  scores_val_pca, scores_test_pca, scores_val_lda, scores_test_lda = [], [], [] ,[]
  for train_idx, valid_idx in kf.split(X_trainval):

    # Récupération du fold 
    X_train_fold, X_val_fold = X_trainval[train_idx], X_trainval[valid_idx]
    y_train_fold, y_val_fold = y_trainval[train_idx], y_trainval[valid_idx]

    # Utilisation de l'algorithme PCA
    n_components_PCA = 150
    pca = PCA(n_components=n_components_PCA).fit(X_train_fold) # PCA training

    # Utilisation de l'algorithme LDA
    n_components_LDA = 5
    lda = LDA(n_components=n_components_LDA).fit(X_train_fold, y_train_fold) # LDA training

    X_train_pca = pca.transform(X_train_fold)  # PCA transformation 
    X_val_pca = pca.transform(X_val_fold)      # PCA transformation 
    X_test_pca = pca.transform(X_test)    # PCA transformation 

    X_train_lda = lda.transform(X_train_fold)  # LDA transformation 
    X_val_lda = lda.transform(X_val_fold)      # LDA transformation 
    X_test_lda = lda.transform(X_test)    # LDA transformation 

    # Linear SVM 
    cls_pca = LinearSVC(C=C).fit(X_train_pca, y_train_fold) # Linear SVM training
    cls_lda = LinearSVC(C=C).fit(X_train_lda, y_train_fold) # Linear SVM training
    
    scores_val_pca.append(cls_pca.score(X_val_pca, y_val_fold))   # Stockage des scores de classification
    scores_test_pca.append(cls_pca.score(X_test_pca, y_test))   # Stockage des scores de classification
    scores_val_lda.append(cls_lda.score(X_val_lda, y_val_fold))   # Stockage des scores de classification
    scores_test_lda.append(cls_lda.score(X_test_lda, y_test))   # Stockage des scores de classification

  # On récupère le score moyen du SVM pour une valeur de C donnée lorsque testé pour les 4 folds
  val_mean_score_over_folds_pca = sum(scores_val_pca)/len(scores_val_pca)
  test_mean_score_over_folds_pca = sum(scores_test_pca)/len(scores_test_pca)
  val_mean_score_over_folds_lda = sum(scores_val_lda)/len(scores_val_lda)
  test_mean_score_over_folds_lda = sum(scores_test_lda)/len(scores_test_lda)
  print(f"C Value : {C} ")
  print(f"  ===> Validation classification score PCA : {val_mean_score_over_folds_pca}")
  print(f"  ===> Test classification score PCA : {test_mean_score_over_folds_pca}")
  print(f"  ===> Validation classification score LDA : {val_mean_score_over_folds_lda}")
  print(f"  ===> Test classification score LDA : {test_mean_score_over_folds_lda}")

C Value : 0.001 
  ===> Validation classification score PCA : 0.6281320844767928
  ===> Test classification score PCA : 0.7461240310077519
  ===> Validation classification score LDA : 0.6902786063715547
  ===> Test classification score LDA : 0.7034883720930232
C Value : 0.01 
  ===> Validation classification score PCA : 0.7877460923517481
  ===> Test classification score PCA : 0.8391472868217055
  ===> Validation classification score LDA : 0.6971811239708865
  ===> Test classification score LDA : 0.7209302325581395
C Value : 0.1 
  ===> Validation classification score PCA : 0.8274340770791075
  ===> Test classification score PCA : 0.8527131782945737
  ===> Validation classification score LDA : 0.6911287435866841
  ===> Test classification score LDA : 0.7267441860465116


c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.w

C Value : 1 
  ===> Validation classification score PCA : 0.7980998687507457
  ===> Test classification score PCA : 0.8313953488372093
  ===> Validation classification score LDA : 0.6859682615439686
  ===> Test classification score LDA : 0.7151162790697674


c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.w

C Value : 5 
  ===> Validation classification score PCA : 0.7825736785586446
  ===> Test classification score PCA : 0.7926356589147286
  ===> Validation classification score LDA : 0.6816400190908006
  ===> Test classification score LDA : 0.7073643410852712


...

c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.w

C Value : 100 
  ===> Validation classification score PCA : 0.767888676768882
  ===> Test classification score PCA : 0.7945736434108527
  ===> Validation classification score LDA : 0.6678469156425247
  ===> Test classification score LDA : 0.6937984496124031


c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.w

C Value : 1000 
  ===> Validation classification score PCA : 0.7592620212385157
  ===> Test classification score PCA : 0.7926356589147288
  ===> Validation classification score LDA : 0.6635216561269538
  ===> Test classification score LDA : 0.6976744186046512


c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\vicri\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best pour c= 0.1 avec PCA ou LDA pour linear SVM

In [8]:
from sklearn.svm import SVC

C = [0.001, 0.01, 0.1, 1, 5, 10, 100, 1000]


best_test_score_pca = {"poly": 0, "rbf": 0} # Best test classification score for poly and rbf 
best_c_value_pca = {"poly": 0, "rbf": 0}
best_test_score_lda = {"poly": 0, "rbf": 0} # Best test classification score for poly and rbf 
best_c_value_lda = {"poly": 0, "rbf": 0}

# PCA ou LDA puis SVM polynomial ou SVM RBF
for kern in ['poly', 'rbf']:
  for c in C : 
    scores_val_pca, scores_test_pca, scores_val_lda, scores_test_lda = [], [], [] ,[]
    for train_idx, valid_idx in kf.split(X_trainval):
      # Récupération du fold 
      X_train_fold, X_val_fold = X_trainval[train_idx], X_trainval[valid_idx]
      y_train_fold, y_val_fold = y_trainval[train_idx], y_trainval[valid_idx]

      # Utilisation de l'algorithme PCA
      n_components = 50
      pca = PCA(n_components=n_components).fit(X_train_fold) # PCA training

      # Utilisation de l'algorithme LDA
      n_components_LDA = 5
      lda = LDA(n_components=n_components_LDA).fit(X_train_fold, y_train_fold) # LDA training

      X_train_pca = pca.transform(X_train_fold)  # PCA transformation 
      X_val_pca = pca.transform(X_val_fold)      # PCA transformation 
      X_test_pca = pca.transform(X_test)         # PCA transformation 

      X_train_lda = lda.transform(X_train_fold)  # LDA transformation 
      X_val_lda = lda.transform(X_val_fold)      # LDA transformation 
      X_test_lda = lda.transform(X_test)    # LDA transformation 

      # Kernel SVM 
      clf_pca = SVC(kernel=kern, C=c).fit(X_train_pca, y_train_fold) # SVM training
      clf_lda = SVC(kernel=kern, C=c).fit(X_train_lda, y_train_fold) # SVM training

      scores_val_pca.append(clf_pca.score(X_val_pca, y_val_fold))   # Stockage des scores de classification
      scores_test_pca.append(clf_pca.score(X_test_pca, y_test))   # Stockage des scores de classification
      scores_val_lda.append(clf_lda.score(X_val_lda, y_val_fold))   # Stockage des scores de classification
      scores_test_lda.append(clf_lda.score(X_test_lda, y_test))   # Stockage des scores de classification

    # On récupère le score moyen du SVM pour une valeur de C donnée lorsque testé pour les 4 folds
    val_mean_score_over_folds_pca = sum(scores_val_pca)/len(scores_val_pca)
    test_mean_score_over_folds_pca = sum(scores_test_pca)/len(scores_test_pca)
    val_mean_score_over_folds_lda = sum(scores_val_lda)/len(scores_val_lda)
    test_mean_score_over_folds_lda = sum(scores_test_lda)/len(scores_test_lda)
    if test_mean_score_over_folds_pca > best_test_score_pca[kern] : 
      best_test_score_pca[kern] = test_mean_score_over_folds_pca
      best_c_value_pca[kern] = c
    if test_mean_score_over_folds_lda > best_test_score_lda[kern] : 
      best_test_score_lda[kern] = test_mean_score_over_folds_lda
      best_c_value_lda[kern] = c
for kern in ['poly', 'rbf']:
  print(f"Best {kern} classification score is {best_test_score_pca[kern]}, for PCA obtained for C = {best_c_value_pca[kern]} ")
  print(f"Best {kern} classification score is {best_test_score_lda[kern]}, for LDA obtained for C = {best_c_value_lda[kern]} ")

Best poly classification score is 0.7228682170542635, for PCA obtained for C = 10 
Best poly classification score is 0.7073643410852712, for LDA obtained for C = 1 
Best rbf classification score is 0.8372093023255813, for PCA obtained for C = 5 
Best rbf classification score is 0.7286821705426356, for LDA obtained for C = 0.1 


### Convolutional Neural Network

In [9]:
import torch
import torchvision
import torch.nn as nn #(le package neural network)
import torch.nn.functional as F #le package neural network fonctionnel
import torch.optim as optim
from torchvision import datasets, transforms

### Importation de la base de données ###

nb_classes = 5749

# Nombre de batch d'entraînement et de tests = Nombres d'images passées au CNN avant chaque actualisation des poids du modèle.
batch_size_train = 100
batch_size_test = 100

# Load de la partie train de la base de données LFWPeople
train_lfw_dset =  torchvision.datasets.LFWPeople('./files/',  split='train', download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

# Dataloader d'entraînement - Permettra de donner les données au modèle par batch (Lot) de 2 images
train_loader = torch.utils.data.DataLoader(train_lfw_dset,
  batch_size=batch_size_train, shuffle=True)

# Load de la partie test de la base de données LFWPeople
test_lfw_dset =   torchvision.datasets.LFWPeople('./files/', split='test', download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

# Dataloader de test - Permettra de donner les données au modèle par batch (Lot) de 2 images
test_loader = torch.utils.data.DataLoader(test_lfw_dset,
  batch_size=batch_size_test, shuffle=True)

gpu = torch.device("cuda") # Définition du GPU 

Files already downloaded and verified
Files already downloaded and verified


#### Training du modele

In [10]:
# Définition d'un CNN 2 couches de convolution

class MonCNN(torch.nn.Module):
  def __init__(self, nombre_de_classes):
    super(MonCNN, self).__init__()
    self.conv1 = nn.Conv2d(3, 32, 3)          # Couche de convolution - Paramètres : (Nombre de canaux de l'input (1), nombre de canaux de l'output (32), Kernel size (3*3))
    self.relu1 = nn.ReLU()                    # Fonction d'activation 
    self.max_pool1 = nn.MaxPool2d(2, 2)       # Couche maxpooling - Paramètres : (taille du filtre = : 2*2, stride (2*2) )
    self.conv2 = nn.Conv2d(32, 64, 3)         # Augmenter la taille du kernel size pour capturer des motifs plus grands
    self.relu2 = nn.ReLU()                    # Ajouter une deuxième fonction d'activation
    self.max_pool2 = nn.MaxPool2d(2, 2)       # Ajouter une deuxième couche maxpooling
    self.fc1 = nn.Linear(64 * 61 * 61, 256)     # Adapter la taille de la couche entièrement connectée en fonction de la taille du feature map après la deuxième couche de convolution
    self.relu3 = nn.ReLU()                    # Ajouter une fonction d'activation après la couche FC
    self.fc2 = nn.Linear(256, nombre_de_classes)  # Couche FC - Paramètres : (Nombre de canaux du vecteur input (256), Taille du vecteur output (Nombres de classes) )
  def forward(self, x): # Dans la partie forward, on passe l'image / le lot (=batch) d'images dans chaque couche précédemment définies
    out = self.conv1(x)
    out = self.relu1(out)
    out = self.max_pool1(out)
    out = self.conv2(out)
    out = self.relu2(out)
    out = self.max_pool2(out)
    out = out.view(out.size(0), -1)  # Flatten (=> Transformer le feature map en vecteur ligne pour donner ensuite à la couche FC)
    out = self.fc1(out)
    out = self.relu3(out)            # Ajouter une fonction d'activation après la couche FC
    out = self.fc2(out)
    return out

mon_model = MonCNN(nb_classes) # On précise que le nombre de classes
mon_model.cuda()
mon_optimizer = torch.optim.SGD(mon_model.parameters(), lr=0.1) # Stochastic Gradient Descent (SGD) 

nb_train_data = len(train_lfw_dset.targets)
# On entraîne le modèle en itérant sur les batch de données 
for i, one_batch in enumerate(train_loader):
  if (i+1) % 100 == 0: # Simple affichage de temps à autre pour savoir où en est rendue la boucle
    print(f"Batch n° {i+1} / {int(nb_train_data/batch_size_train)}")
  mon_optimizer.zero_grad()       # Réinitialisation du gradient
  data, ground_truth = one_batch  # Récupération des 10 images et des 10 labels contenues dans le batch 

  # Transfert des données sur GPU 
  data = data.cuda()
  ground_truth = ground_truth.cuda()

  output_de_model = mon_model(data)  # Extraction du vecteur caractéristique par le modèle
  loss = F.cross_entropy(output_de_model, ground_truth) # Calcul de la fonction de pertes
  loss.backward()   # Backpropagation 
  mon_optimizer.step()  # Actualisation des poids du modèle

#### Test et précision 

In [11]:
nb_test_data = len(test_lfw_dset.targets)
correct = 0

# Phase de test
with torch.no_grad(): # On ne veut pas calculer de gradient puisque le réseau est entraîné 
    # On test le modèle en itérant sur les batch de données de test
    for idx, (data, ground_truth) in enumerate(test_loader): 
      if (idx +1) % 50 == 0 : 
        print(f"Batch n° {idx+1} / {int(nb_test_data/batch_size_test)}")

      # Transfert des données sur GPU 
      data = data.cuda()
      ground_truth = ground_truth.cuda()

      output = mon_model(data) # Extraction des features 
      pred = output.data.max(1, keepdim=True)[1] # Récupération de la prediction du modèle
      correct += pred.eq(ground_truth.data.view_as(pred)).sum().item() # Vérification de la prédiction du modèle

# Précision du modèle
100*correct / len(test_loader.dataset) #taux de bonne classification 

0.0

#### Resnet18

In [12]:
import torchvision.models as models
gpu = torch.device("cuda") # Définition du GPU
nb_train_data = len(train_lfw_dset.targets)

resnet18 = models.resnet18(pretrained=True)
resnet18.fc = nn.Linear(resnet18.fc.in_features, nb_classes) 
resnet18.cuda() # Place le modèle sur le GPU (Calculs plus rapides)

mon_optimizer = torch.optim.SGD(resnet18.parameters(), lr=0.01) # stochastic gradient descent (SGD) 
for i, (data, ground_truth) in enumerate(train_loader):
  if (i+1) % 500 == 0:
    print(f"Batch n° {i+1} / {int(nb_train_data/batch_size_train)}")
    print(loss)
  mon_optimizer.zero_grad()
  data, ground_truth = data.cuda(), ground_truth.cuda()
  data = data.expand(-1, 3, -1, -1) # Grey image was defined on 1 channel. Now defined on 3 channels as an RGB image would be. 
  output_de_model = resnet18(data)
  print(loss)
  loss = F.cross_entropy(output_de_model, ground_truth.cuda())
  loss.backward()
  mon_optimizer.step()

tensor(8.6474, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.7845, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.8702, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.9340, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.8039, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.8074, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.7472, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.8040, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.7646, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.7881, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.7766, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.6638, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.6260, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.5960, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.6290, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.5477, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.6440, device='cuda:0', grad_fn=

In [13]:
print((train_lfw_dset))
print(len(test_lfw_dset.targets))


Dataset LFWPeople
    Number of datapoints: 9525
    Root location: ./files/lfw-py
    Alignment: funneled
    Split: train
    Classes (identities): 5749
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )
3708


In [14]:
correct = 0

with torch.no_grad():
    for data, ground_truth in test_loader:
      data = data.expand(-1, 3, -1, -1) # Grey image was defined on 1 channel. Now defined on 3 channels as an RGB image would be. 
      output = resnet18(data.cuda())
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(ground_truth.cuda().data.view_as(pred)).sum().item()

print(100*correct / len(test_loader.dataset))

0.0
